# Tools for Modeling: The Brain Modeling Toolkit and SONATA data format

#### January 28, 2021

This tutorial will show how to build and simulate a toy model of the mouse V1 cortical area network using the Brain Modeling Toolkit (BMTK). Including how to modify and simulate larger more realistic visual cortical models released by the Allen Institute.

The network uses biophysically realistic multi-comparment cell models, but small enough to run on most linux/windows/osx computers and laptops. 

For more information about cortical modeling and simulation at the Allen Institute:
https://portal.brain-map.org/explore/models

For more information and tutorials on using the BMTK:
https://alleninstitute.github.io/bmtk



## Installing BMTK
\[https://alleninstitute.github.io/bmtk/installation.html\]

BMTK will require python 2.7 or 3.6+. You can clone the latest developement branch from https://github.com/AllenInstitute/bmtk.git. Alternatively one can install using python's pip utility:

```bash
 $ pip install bmtk
```

Or if you're using a CONDA environment:

```bash
 $ conda install -c kaeldai bmtk
```

For simulation [NEURON](https://www.neuron.yale.edu/neuron/) will also be required to be installed (including python bindings). To quickly install NEURON on your machine:
```bash
 $ pip install neuron
```

or 
```
 $ conda install -c conda-forge neuron
```


#### The BMTK Docker Image

We have included a docker image that will come with BMTK, neuron, and a ready-to-use jupyter notebook server with this tutorial and it's content already installed. To run the docker image

```bash
 ## Linux or OSX
 $ docker run -v $(pwd):/home/shared/workspace -p 8888:8888 alleninstitute/bmtk jupyter
 
 ## Windows PowerShell
 $ docker run -v ${PWD}:/home/shared/workspace -p 8888:8888 alleninstitute/bmtk jupyter
```

Then open a web browser to __127.0.0.1:8888/__. This current tutorial can be found under the folder _tutorials/modeling_tut_2021_/Mouse_L4.ipynb.

## Building the Network

#### 1. Building the cells

We want to create a simple 300 cell network modeling using cortical models from the mouse primary visual (V1) area. The first step is figuring out different cell types and parameters we want to use in our network. When we know that we can begin building our network model first by initializing the network using _bmtk.builder.networks.NetworkBuilder_ plus calls to _add_nodes()_ method

In [1]:
import numpy as np
from bmtk.builder.networks import NetworkBuilder

# intialize a new population of cells called "V1"
v1 = NetworkBuilder('V1')

# Add a population of 80 "Scnn1a" type nodes
v1.add_nodes(
    N=80,   
    
    # Reserved SONATA keywords used during simulation
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    dynamics_params='472363762_fit.json',
    morphology='Scnn1a_473845048_m.swc',
    
    # The x, y, z locations of each cell in a column
    x=np.random.normal(0.0, 20.0, size=80),
    y=np.random.uniform(400.0, 500.0, size=80),
    z=np.random.normal(0.0, 20.0, size=80),
    
    # Euler rotations of the cells
    rotation_angle_xaxis=np.random.uniform(0.0, 2*np.pi, size=80),
    rotation_angle_yaxis=np.random.uniform(0.0, 2*np.pi, size=80),
    rotation_angle_zaxis=-3.646878266,
    
    # Optional parameters
    tuning_angle=np.linspace(start=0.0, stop=360.0, num=80, endpoint=False),
    pop_name='Scnn1a',
    location='L4',
    ei='e',
)

The above call to add_nodes() will and __N=80__ different individual cells to our network. 

The attributes __model_type__, __model_template__, __dynamics_params__, and __morphology__ are SONATA reserved keywords that will instruct BMTK (or other SONATA compliant tools) how to build cell models during the simulation. In this model we are using biophysically realistic models, with dynamical parameters (.json) and morphology (.swc) that we can download from the [Allen Cell Types Database](https://celltypes.brain-map.org/data). We have already downloaded the required json and swc files and have saved them under _components/biophysical_neuron_templates_ and 
_components/morphologies/_, respectively.

Attributes __x__, __y__, __z__ are used as the euclidian locations of the cells, while the __rotation_angle__ parameters are used to orientat to rotations of cells. Note that for the x and y rotations we are passing in an array of size N, but the z rotation is defined by a scalar. SONATA and the BMTK Network Builder allows us to differentiate between attributes that are unique to each cell versus shared attributes. Each of the 80 cells will have unique x and y rotations, but share the same pre-calculated z rotatation

The final remaining attributes are not reserved SONATA keywords, but rather optional attributes that will be helpful during analysis. The __tuning_angle__ property (how well a cell is tuned for different stimulus grating) is an attribute shown is excitatory cells of the visual cortex and thus wouldn't be applicable to other models or even non-exciatory cells-types

We now can ad other cell-types to our model, including inhibitory PV (parvalbumin) cells, with future calls to __add_nodes__:

In [2]:
v1.add_nodes(
    # Rorb excitatory cells
    N=80, pop_name='Rorb', location='L4', ei='e',
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    dynamics_params='473863510_fit.json',
    morphology='Rorb_325404214_m.swc',
    x=np.random.normal(0.0, 20.0, size=80),
    y=np.random.uniform(400.0, 500.0, size=80),
    z=np.random.normal(0.0, 20.0, size=80),   
    rotation_angle_xaxis=np.random.uniform(0.0, 2*np.pi, size=80),
    rotation_angle_yaxis=np.random.uniform(0.0, 2*np.pi, size=80),
    rotation_angle_zaxis=-4.159763785,
    tuning_angle=np.linspace(start=0.0, stop=360.0, num=80, endpoint=False),
)


v1.add_nodes(
    # Nr5a1 excitatory cells
    N=80, pop_name='Nr5a1', location='L4', ei='e',
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    dynamics_params='473863035_fit.json',
    morphology='Nr5a1_471087815_m.swc',
    x=np.random.normal(0.0, 20.0, size=80),
    y=np.random.uniform(400.0, 500.0, size=80),
    z=np.random.normal(0.0, 20.0, size=80),   
    rotation_angle_xaxis=np.random.uniform(0.0, 2*np.pi, size=80),
    rotation_angle_yaxis=np.random.uniform(0.0, 2*np.pi, size=80),
    rotation_angle_zaxis=-4.159763785,
    tuning_angle=np.linspace(start=0.0, stop=360.0, num=80, endpoint=False),
)

v1.add_nodes(
    # Parvalbuim inhibitory cells, note these don't have a tuning angle and ei=i
    N=60, pop_name='PV1', location='L4', ei='i',
    model_type='biophysical',
    model_template='ctdb:Biophys1.hoc',
    dynamics_params='472912177_fit.json',
    morphology='Pvalb_470522102_m.swc',
    x=np.random.normal(0.0, 20.0, size=60),
    y=np.random.uniform(400.0, 500.0, size=60),
    z=np.random.normal(0.0, 20.0, size=60),   
    rotation_angle_xaxis=np.random.uniform(0.0, 2*np.pi, size=60),
    rotation_angle_yaxis=np.random.uniform(0.0, 2*np.pi, size=60),
    rotation_angle_zaxis=-2.539551891
)


#### 2. Creating the connections 

Now that we have our cells/nodes, we can go ahead and create the synapse/connectivity matrix using the __add_edges__ method. We will want to call this method multiple times to add different connection types with different properties and rules. For each connection type we must select what subpopulation of cells to use for the source (pre-synaptic) and target (post-synaptic) cells. We can select our cells using any combination of attributes

In [3]:
v1.add_edges(
    ## Inh --> Exc type connections.
    source={'ei': 'i'}, 
    target={'ei': 'e'},
    
    # Each source (i) cell will connect once to every target (e) cell
    connection_rule=1,
    
    # SONATA keywords 
    syn_weight=0.0002,
    delay=2.0,
    dynamics_params='InhToExc.json',
    model_template='exp2syn',
    
    # Special BMTK attributes specifying where to set target synape location
    distance_range=[0.0, 1e+20],
    target_sections=['somatic', 'basal']
)


v1.add_edges(
    ## Exc --> Inh type connections.
    source={'ei': 'i'}, 
    target={'ei': 'e'},
    
    connection_rule=1,
              
    syn_weight=0.00018,
    delay=2.0,
    dynamics_params='ExcToInh.json',
    model_template='exp2syn',
    distance_range=[0.0, 50.0],
    target_sections=['somatic', 'basal', 'apical'],
)



The __connection_rule__ parameter is used to determine the number of connections/synapses between each source and target cell. In the simplist case like above we can pass in a scalar value. You may also pass in a matrix of shape T x S (T=# or targets, S=# of sources), or use in a custom function.

With a user defined function it will pass in a source and target, which can be used as dictionaries to access cell parameters, and expects to return the number of connections
```python
def <my_connection_rule>(source, target, **opt_params):
    return N
```

For example for Inh --> Inh connection we want there to be no connections if the cells have the same id, otherwise randomly select the number of synapses

In [4]:
def ignore_autopases(source, target):
    # No synapses if source == target, otherwise randomize
    if source['node_id'] == target['node_id']:
        return 0
    else:
        return np.random.randint(0, 6)

v1.add_edges(
    ## Inh --> Inh type connections.
    source={'ei': 'i'}, 
    target={'ei': 'e'},
    
    connection_rule=ignore_autopases,
              
    syn_weight=0.00015,
    delay=2.0,
    dynamics_params='InhToInh.json',
    model_template='exp2syn',
    distance_range=[0.0, 1.0e+20],
    target_sections=['somatic', 'basal']
)


For Exc --> Exc we want to test an theory that cells with closer __tuning_angle__ are more likely to be connected with each other:

In [5]:
def tunning_angle(source, target, max_syns):
    # ignore autoapses
    if source['node_id'] == target['node_id']:
        return 0
    
    # num of synapses is higher the closer the tuning_angles
    src_tuning = source['tuning_angle']
    trg_tuning = target['tuning_angle']
    dist = np.abs((src_tuning - trg_tuning + 180) % 360 - 180)
    p_dist = 1.0 - (np.max((dist, 10.0)) / 180.0)
    return np.random.binomial(n=max_syns, p=p_dist)
    
    

v1.add_edges(
    source={'ei': 'e'}, 
    target={'ei': 'e'},
    
    connection_rule=tunning_angle,
    connection_params={'max_syns': 9}, # pass in options to tuning_angle function
    
    syn_weight=6.4e-05,
    delay=2.0,
    weight_function='weighted_distance',
    distance_range=[30.0, 150.0],
    target_sections=['basal', 'apical'],
    dynamics_params='ExcToExc.json',
    model_template='exp2syn'
)

Other connection parameters, particularly __syn_weight__ and __delay__, are scalar values and will be the same for each connection. There are multiple ways to set such values so it is customized for each connection. We will show one way of doing so later during the simulation stage.

#### 3. Building the network

Finally we want to build our network and save the network files in SONATA format into the _network/_ directory. BMTK will intuit the names of the files.

In [6]:
v1.build()
v1.save(output_dir='network')

#### 4. External Network

BMTK and SONATA gives modelers the options to build different parts of a network piecewise. For example we may want to build models of thalamacortical and higher cortical areas that we can connect to our V1 model.

Next we create a separate network of the LGN, an flat area of the Thalamus with exciatory cells that project onto the V1. As before we define our cells, connect them to the v1 network, then build and save them as SONATA files in the _network/_ directory.

In [11]:
lgn = NetworkBuilder('LGN')  # Initialize network called 'lgn'

lgn.add_nodes(
    N=50, 
    model_type='virtual',
    model_template='lgnmodel:tOFF_TF15',
    x=np.random.uniform(0.0, 240.0, 50),
    y=np.random.uniform(0.0, 120.0, 50),
    spatial_size=1.0,
    dynamics_params='tOFF.json'
)

lgn.add_nodes(
    N=50, 
    model_type='virtual',
    model_template='lgnmodel:tON',
    x=np.random.uniform(0.0, 240.0, 50),
    y=np.random.uniform(0.0, 120.0, 50),
    spatial_size=1.0,
    dynamics_params='tON.json'
)

lgn.add_edges(
    source=lgn.nodes(), # Select all LGN cells as the sources
    target=v1.nodes(ei='e'), # select V1 exc cells are the target
              
    connection_rule=lambda *_: np.random.randint(1, 5),
    syn_weight=4e-03,
    delay=2.0,    
    distance_range=[0.0, 150.0],
    target_sections=['basal', 'apical'],
    dynamics_params='LGN_ExcToExc.json',
    model_template='exp2syn'
)

lgn.add_edges(
    source=lgn.nodes(), # Select all LGN cells as the sources
    target=v1.nodes(ei='i'), # select V1 exc cells are the target
              
    connection_rule=lambda *_: np.random.randint(1, 5),
    syn_weight=4e-03,
    delay=2.0,    
    distance_range=[0.0, 150.0],
    target_sections=['basal', 'apical'],
    dynamics_params='LGN_ExcToInh.json',
    model_template='exp2syn'
)

lgn.build()
lgn.save(output_dir='network')


Unlike the V1 cells, the LGN cells are __model_type=virtual__. In SONATA _virtual_ cells are special type of cell models that can't be recorded from, but can be used to synapse onto and stimulate other cells. There are essentially place-holders for spike trains (that will be set during the simulation).

we have given the lgn special dynamics_params and model_template values, however these won't be used during the V1 simulation. There parameter values are used by FilterNet module.

# Simulation of the Network

Now that we have built the SONATA network models files (or recieved the files from another source), we can go ahead and setup a simulation to run. Before we can do that there are still a few things that need to be taken care of, including setting up run-time parameters, choosing the stimulus, and deciding what to record.

### Simulation Directory Structure

Almost all of the parameters required to initialize and run a full V1 simulation will be stored in a [SONATA base json configuration file](https://github.com/AllenInstitute/sonata/blob/master/docs/SONATA_DEVELOPER_GUIDE.md#tying-it-all-together---the-networkcircuit-config-file), in this case named _config.bionet.json_. If you need to (re)create the file the following command will create a template:

```python
  python -m bmtk.utils.sim_setup  \
   --network network              \
   --dt 0.1                       \
   --tstop 3000.0                 \  
   --compile-mechanisms           \
   --include-examples             \
   --config=config.bionnt.json    \
   bionet .
```

Opening up the json with a text editor we can see the configuration is divided into different sections


#### networks

This section contains paths to the sonata network files we built above. We can use this to add or remove node populations and edges before running another simulation.

#### components

Contains the paths to the different external files and parameters required to run a simulation; morphologies, electricophysicological parameters (eg. NeuroML files), synaptic model parameters, etc. Files that are not directly defined as a part of the SONATA files but used by specific simulator software (eg NEURON, NEST).

The required files for this tutorial has already be created. In other cases one would likely have to spend a signficant amount of time creating the optimized parameter files.


#### run

Run-time parameters and conditions, including simulation time and the time-step (all time values are in miliseconds).

#### output

Paths and parameters used for simulation output and logging.



#### inputs

This section can be used to specify one or more stimulus we want to use on our V1 model. Depending on the type of cell models being used, we can use spike trains, current clamps, voltage clamps, or extracellular electrodes.  

For our V1 network we created a population of LGN "virtual" cells that synapse onto our V1 cells and drive them with spike trains. All that is missing is a "spike-trains" file, which can be either a CSV, NWB, or SONATA HDF5 file. BMTK includes way to create spike-train files.

The following will create a series of spike trains for the LGN cells using a Poisson distrubtion of 15 Hz:

In [16]:
from bmtk.utils.reports.spike_trains import PoissonSpikeGenerator

psg = PoissonSpikeGenerator(population='LGN')
psg.add(
    node_ids=range(0, 100),
    firing_rate=15.0,    # 15 Hz, we can also pass in a nonhomoegenous function/array
    times=(0.0, 3.0)    # Firing starts at 0 s up to 3 s
)

psg.to_sonata('inputs/lgn_spikes.h5')
psg.to_csv('inputs/lgn_spikes.csv')

and update the "inputs" section 

```json
    "LGN_spikes": {
        "input_type": "spikes",
        "module": "h5",
        "input_file": "./inputs/lgn_spikes.nwb",
        "node_set": "LGN"
    }
```

BMTK also includes a tool FilterNet that allows us to turn movies and images into firing rates using optimized filter models, which we will do in future sections.


#### reports

By default BMTK will record the V1 cell spikes into a file specified under _outputs/spikes_file_. For these types of models we can also record other parameters like membrane potential, calcium fluxuation, or extracellular electrical fields by adding to the "reports" section of the configuration file. 

For example the following can be used to record 

### Running the simulation

from command line
...

from the code


In [ ]:

from bmtk.simulator import bionet


conf = bionet.Config.from_json('sim_ch04/simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

### Adjust the Connection weights

In [ ]:
def gaussianLL(edge_props, source, target):
    src_tuning = source['tuning_angle']
    tar_tuning = target['tuning_angle']
    w0 = edge_props["syn_weight"]
    sigma = edge_props["weight_sigma"]

    


In [ ]:
def tunning_angle(source, target, max_syns):
    # ignore autoapses
    if source['node_id'] == target['node_id']:
        return 0
    
    # num of synapses is higher the closer the tuning_angles
    src_tuning = source['tuning_angle']
    trg_tuning = target['tuning_angle']
    dist = np.abs((src_tuning - trg_tuning + 180) % 360 - 180)
    p_dist = 1.0 - (np.max((dist, 10.0)) / 180.0)
    return np.random.binomial(n=max_syns, p=p_dist)


N = 20
for i, angle in zip(range(N), np.linspace(0.0, 360.0, N)):
    syns = tunning_angle(
        source={'node_id': 0, 'tuning_angle': 0.0},
        target={'node_id': 1, 'tuning_angle': angle},
        max_syns=9
    )
    print(angle, syns)